# Evaluate model using Giskard

In this notebook we leverage the Giskard framework to evaluate our agent's ability to extract information and summarize context to answer a query.

## Dependencies

In [1]:
# %pip install "giskard[llm]" --upgrade
# %pip install langchain langchain-openai langchain-community pypdf faiss-cpu openai tiktoken?

In [2]:
# load env vars
import os
from dotenv import load_dotenv
load_dotenv('giskard_evaluation.env', override=True)  # change project evaluation name in environment

if os.environ.get('OPENAI_API_KEY', ''):
    print('OPENAI_API_KEY is set')
print('Langchain project:', os.environ.get('LANGCHAIN_PROJECT', ''))

OPENAI_API_KEY is set
Langchain project: ai-librarian-giskard-evaluation


## Build Agent

In [3]:
# create an agent
from ai_librarian import AiLibrarian

avatar = AiLibrarian()

avatar.create_worker_agent()

# test the agent
avatar.generate_model_answer('The Three Musketeers')

c:\Users\Nath\anaconda3\envs\ironhack\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
c:\Development\_repos\Ironhack\Projects\ik-multimodal-ai-librarian\models.py:101: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferWindowMemory(




> Entering new AgentExecutor chain...

Invoking: `search_database_for_book_information` with `{'query_text': 'The Three Musketeers'}`




Batches:   0%|          | 0/1 [00:00<?, ?it/s]

{'documents': [["The Three Musketeers (A Stepping Stone Book(TM)) - In seventeenth-century France, young D'Artagnan befriends three musketeers, Athos, Porthos, and Aramis, and joins them in trying to outwit the enemies of the king and queen. Simultaneous. -  Juvenile Fiction , Action & Adventure , General", "The Three Musketeers is a classic novel by Alexandre Dumas, published in 1844, which follows the adventures of d'Artagnan, a young man who travels to Paris to join the Musketeers of the Guard. He befriends three musketeers—Athos, Porthos, and Aramis—and together they engage in various escapades involving political intrigue, romance, and duels. The story is set in the 17th century and explores themes of friendship, loyalty, and honor, as d'Artagnan navigates the complexities of court life and battles against the schemes of the villainous Cardinal Richelieu and Milady de Winter. The novel is renowned for its vibrant characters and thrilling plot, making it a timeless adventure tale."

'"The Three Musketeers," written by Alexandre Dumas and published in 1844, is a thrilling adventure that immerses readers in the court politics of 17th-century France. The story revolves around a young man named d\'Artagnan who befriends the three titular musketeers—Athos, Porthos, and Aramis—as they engage in a series of exciting escapades filled with themes of friendship, loyalty, and honor in the face of deception orchestrated by figures like Cardinal Richelieu and Milady de Winter. The novel is celebrated for its vibrant characters and the spirit of camaraderie that resonates through poignant moments of adventure and conflict.'

## Create Dataset
Our agent is not a QnA-oriented one and we don't have a dataset of questions/answers to evaluate if it's answering correctly. But we can evaluate its summarizing abilities and use a dataset of book titles/descriptions as QnA.

In [4]:
import pandas as pd
# load 100 books from the dataset we loaded into the database

# load the clean dataset into a dataframe
books_df = pd.read_csv('datasets/BooksDatasetClean.csv')

# sample a smaller dataset with only book titles and reviews
n_titles = 1000
titles = books_df.sample(n=n_titles)['Title'].tolist()

# get description for each title
descriptions = []
for title in titles:
    descriptions.append(books_df[books_df['Title'] == title]['Description'].values[0])

# create a dataframe with the titles and descriptions
df_titles = pd.DataFrame({'query': titles, 'answer': descriptions})

df_titles.head()

,query,answer
0,Rudolph the Red-Nosed Reindeer Shines Again (A...,A retelling of the story of the reindeer with ...
1,Woman Hollering Creek: And Other Stories,"A collection of stories by Sandra Cisneros, th..."
2,Jumping the Curve: Innovation and Strategic Ch...,Foreword by Tom PetersInternationally known ma...
3,"Half Full: Meditations on Hope, Optimism and t...",Half Full is an invitation to take a pause. To...
4,The Opposite of Love: A Novel (Random House Re...,With perfect pitch for the humor and heartbrea...


## Wrap up agent and dataset in Giskard

In [5]:
import giskard
import pandas as pd

def model_predict(df: pd.DataFrame):
    """Wraps the LLM call in a simple Python function.

    The function takes a pandas.DataFrame containing the input variables needed
    by your model, and must return a list of the outputs (one for each row).
    """
    return [avatar.generate_model_answer(query) for query in df["query"]]

giskard_model = giskard.Model(
    model=model_predict,
    model_type="text_generation",
    name="Book Description Generator",
    description="This model answer questions about books, relying on gpt-4o-mini and tools to retrieve information from a vectorstore with book information, wikipedia and youtube transcripts.",
    feature_names=["query"],
)

In [10]:
# test model on one sample

# get random sample
row = df_titles.sample(1)
query, answer = row['query'].values[0], row['answer'].values[0]

examples = [
    query,
]
giskard_dataset = giskard.Dataset(pd.DataFrame({"query": examples}), target=None)

print(giskard_model.predict(giskard_dataset).prediction)



> Entering new AgentExecutor chain...

Invoking: `search_database_for_book_information` with `{'query_text': 'The Mighty Big Book of Riddles'}`




Batches:   0%|          | 0/1 [00:00<?, ?it/s]

{'documents': [['The Mighty Big Book of Riddles (Mighty Big Books) - Each page of The Mighty Big Book of Jokes and The Mighty Big Book of Riddles is jam-packed with visual puns, cartoon gags, illustrations in the margins, and loads and loads of the groan-inducing jokes and riddles that will give any kid fits of hysterical laughter. -  Juvenile Nonfiction , Humor , Jokes & Riddles', 'Biggest Riddle Book in the World - How do you keep a rhinoceros from charging? Take away his credit cards. "Over 2,000 riddles of all kinds, from simple to complex, for all ages. Written by a children\'s librarian, this is one riddle book that should be in both the adult and children\'s collections."--Library Journal. "A real bonanza for riddle fans. A great boost for budding comedians."--School Library Journal. "A massive collection. Index is helpful in finding specific zingers."--Horn Book. -  Juvenile Nonfiction , Humor , Jokes & Riddles', "The Quest for the One Big Thing (A Bug's Life) - A charming book

## Generate short report
Now that our model and QnA dataset are ready, we can ask Giskard to evaluate it and write a report.

In [ ]:
report = giskard.scan(giskard_model, giskard_dataset, only="hallucination")

In [15]:
display(report)

# Save it to a file
report.to_html("evaluation/giscard_scan_hallucination_report.html")

<iframe id="scan-2762992229248" srcdoc="<!doctype html>
<html lang="en">

<head>
 <title>Giskard Scan Results</title>
 <base target="_blank">
 <meta charset="utf-8">
 
<style>pre code.hljs{display:block;overflow-x:auto;padding:1em}code.hljs{padding:3px 5px}
/*!
 Theme: GitHub Dark
 Description: Dark theme as seen on github.com
 Author: github.com
 Maintainer: @Hirse
 Updated: 2021-05-15

 Outdated base version: https://github.com/primer/github-syntax-dark
 Current colors taken from GitHub's CSS
*/.hljs{background:#0d1117;color:#c9d1d9}.hljs-doctag,.hljs-keyword,.hljs-meta .hljs-keyword,.hljs-template-tag,.hljs-template-variable,.hljs-type,.hljs-variable.language_{color:#ff7b72}.hljs-title,.hljs-title.class_,.hljs-title.class_.inherited__,.hljs-title.function_{color:#d2a8ff}.hljs-attr,.hljs-attribute,.hljs-literal,.hljs-meta,.hljs-number,.hljs-operator,.hljs-selector-attr,.hljs-selector-class,.hljs-selector-id,.hljs-variable{color:#79c0ff}.hljs-meta .hljs-string,.hljs-regexp,.hljs-string{color:#a5d6ff}.hljs-built_in,.hljs-symbol{color:#ffa657}.hljs-code,.hljs-comment,.hljs-formula{color:#8b949e}.hljs-name,.hljs-quote,.hljs-selector-pseudo,.hljs-selector-tag{color:#7ee787}.hljs-subst{color:#c9d1d9}.hljs-section{color:#1f6feb;font-weight:700}.hljs-bullet{color:#f2cc60}.hljs-emphasis{color:#c9d1d9;font-style:italic}.hljs-strong{color:#c9d1d9;font-weight:700}.hljs-addition{background-color:#033a16;color:#aff5b4}.hljs-deletion{background-color:#67060c;color:#ffdcd7}.hljs-copy-wrapper{overflow:hidden;position:relative}.hljs-copy-button:focus,.hljs-copy-wrapper:hover .hljs-copy-button{transform:translateX(0)}.hljs-copy-button{background-color:#2d2b57;background-color:var(--hljs-theme-background);background-image:url('data:image/svg+xml;charset=utf-8,<svg xmlns="http://www.w3.org/2000/svg" width="16" height="16" fill="none" viewBox="0 0 24 24"><path fill="%23fff" fill-rule="evenodd" d="M6 5a1 1 0 0 0-1 1v14a1 1 0 0 0 1 1h12a1 1 0 0 0 1-1V6a1 1 0 0 0-1-1h-2a1 1 0 1 1 0-2h2a3 3 0 0 1 3 3v14a3 3 0 0 1-3 3H6a3 3 0 0 1-3-3V6a3 3 0 0 1 3-3h2a1 1 0 0 1 0 2H6Z" clip-rule="evenodd"/><path fill="%23fff" fill-rule="evenodd" d="M7 3a2 2 0 0 1 2-2h6a2 2 0 0 1 2 2v2a2 2 0 0 1-2 2H9a2 2 0 0 1-2-2V3Zm8 0H9v2h6V3Z" clip-rule="evenodd"/></svg>');background-position:50%;background-repeat:no-repeat;border:1px solid #ffffff22;border-radius:.25rem;color:#fff;height:2rem;position:absolute;right:1em;text-indent:-9999px;top:1em;transition:background-color .2s ease,transform .2s ease-out;width:2rem}.hljs-copy-button:hover{border-color:#ffffff44}.hljs-copy-button:active{border-color:#ffffff66}.hljs-copy-button[data-copied=true]{background-image:none;text-indent:0;width:auto}@media (prefers-reduced-motion){.hljs-copy-button{transition:none}}.hljs-copy-alert{clip:rect(0 0 0 0);-webkit-clip-path:inset(50%);clip-path:inset(50%);height:1px;overflow:hidden;position:absolute;white-space:nowrap;width:1px}
/*! tailwindcss v3.3.2 | MIT License | https://tailwindcss.com*/*,:after,:before{border:0 solid #e5e7eb;box-sizing:border-box}:after,:before{--tw-content:""}html{-webkit-text-size-adjust:100%;font-feature-settings:normal;font-family:ui-sans-serif,system-ui,-apple-system,BlinkMacSystemFont,Segoe UI,Roboto,Helvetica Neue,Arial,Noto Sans,sans-serif,Apple Color Emoji,Segoe UI Emoji,Segoe UI Symbol,Noto Color Emoji;font-variation-settings:normal;line-height:1.5;-moz-tab-size:4;-o-tab-size:4;tab-size:4}body{line-height:inherit;margin:0}hr{border-top-width:1px;color:inherit;height:0}abbr:where([title]){-webkit-text-decoration:underline dotted;text-decoration:underline dotted}h1,h2,h3,h4,h5,h6{font-size:inherit;font-weight:inherit}a{color:inherit;text-decoration:inherit}b,strong{font-weight:bolder}code,kbd,pre,samp{font-family:ui-monospace,SFMono-Regular,Menlo,Monaco,Consolas,Liberation Mono,Courier New,monospace;font-size:1em}small{font-size:80%}sub,sup{font-size:75%;line-height:0;position:relative;vertical-align:baseline}sub{bottom:-.25em}sup{top:-.5em}table{border-collapse:

## Run the full report

In [13]:
full_report = giskard.scan(giskard_model, giskard_dataset)

🔎 Running scan…
Estimated calls to your model: ~365
Estimated LLM calls for evaluation: 148

Running detector LLMBasicSycophancyDetector…


> Entering new AgentExecutor chain...

Invoking: `search_database_for_book_information` with `{'query_text': 'Harry Potter'}`




Batches:   0%|          | 0/1 [00:00<?, ?it/s]

{'documents': [["Harry Potter, You're the Best!: A Tribute from Fans the World Over - After St. Martin's published We Love Harry Potter! We'll tell you why we got hundreds of letters from fans around the world who wanted to offer their thoughts about the books they love. Their fertile imaginations have led them to share their creative takes on Harry: their opinions about the plots and characters, their wishes for the next book and the upcoming, highly ancipated movie version, their ideas for playing Harry, Ron, Hermione, and all their other favorite characters, and much more! The collected letters are a delightful testament to how widespread and beloved the Harry Potter phenomenon has become. -  Juvenile Fiction , Fantasy & Magic", "Horrible Harry in Room 2B - Doug discovers that although being Harry's best friend in Miss Mackle's second grade class is not always easy--as Harry revels in doing horrible things--it is always exciting -  Juvenile Fiction , Social Themes , Friendship", 'Ha

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

{'documents': [['The Mighty Big Book of Riddles (Mighty Big Books) - Each page of The Mighty Big Book of Jokes and The Mighty Big Book of Riddles is jam-packed with visual puns, cartoon gags, illustrations in the margins, and loads and loads of the groan-inducing jokes and riddles that will give any kid fits of hysterical laughter. -  Juvenile Nonfiction , Humor , Jokes & Riddles', 'Biggest Riddle Book in the World - How do you keep a rhinoceros from charging? Take away his credit cards. "Over 2,000 riddles of all kinds, from simple to complex, for all ages. Written by a children\'s librarian, this is one riddle book that should be in both the adult and children\'s collections."--Library Journal. "A real bonanza for riddle fans. A great boost for budding comedians."--School Library Journal. "A massive collection. Index is helpful in finding specific zingers."--Horn Book. -  Juvenile Nonfiction , Humor , Jokes & Riddles', "The Quest for the One Big Thing (A Bug's Life) - A charming book

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

The Mars trilogy by Kim Stanley Robinson explores the colonization and terraforming of Mars from 2026 to 2212, focusing on the lives of the First Hundred colonists and their descendants. The series consists of three novels: Red Mars, which details the initial colonization efforts and the ideological conflicts over terraforming; Green Mars, which follows the evolution of Martian society and the struggle for independence from Earth; and Blue Mars, which depicts the long-term results of terraforming and the establishment of a new political order on Mars. The narrative intertwines themes of ecological responsibility, corporate power, and the psychological effects of extreme longevity, as the characters navigate their complex relationships and the challenges of building a new world. The trilogy has received numerous awards and has been influential in the science fiction genre, though adaptations for screen have faced challenges.There are many engaging science fiction books written by women 

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

{'documents': [["Modern African American Writers (Essential Bibliography of American Fiction) - Some of the most profound and original voices in American fiction belong to African American writers: James Baldwin, Charles W. Chesnutt, Ralph Ellison, Zora Neale Hurston, Toni Morrison, Ishmael Reed, Jean Toomer, Alice Walker, and Richard Wright. Their fiction documents one of the largest, oldest and most important cultures in America. Modern African American Writers, part of the Essential Bibliography of American Fiction series, introduces the serious reader and student to each writer's body of work and to sources that illuminate his or her writing and life. Modern African American Writers provides a foreword by Keneth Kinnamon, Ethel Pumphrey Stephens Professor of English at the University of Arkansas, that comments on these writers and discusses their collective contribution to American fiction.Based on Facts On File's four-volume Bibliography of American Fiction series, each volume in 

c:\Users\Nath\anaconda3\envs\ironhack\Lib\site-packages\giskard\scanner\scanner.py:377: UserWarning: 4 errors were encountered while running detectors. Please check the log to understand what went wrong. You can run the scan again with `raise_exceptions=True` to disable graceful handling.
  warning(


In [14]:
display(full_report)

# Save it to a file
full_report.to_html("evaluation/giscard_scan_full_report.html")

<iframe id="scan-2763361236752" srcdoc="<!doctype html>
<html lang="en">

<head>
 <title>Giskard Scan Results</title>
 <base target="_blank">
 <meta charset="utf-8">
 
<style>pre code.hljs{display:block;overflow-x:auto;padding:1em}code.hljs{padding:3px 5px}
/*!
 Theme: GitHub Dark
 Description: Dark theme as seen on github.com
 Author: github.com
 Maintainer: @Hirse
 Updated: 2021-05-15

 Outdated base version: https://github.com/primer/github-syntax-dark
 Current colors taken from GitHub's CSS
*/.hljs{background:#0d1117;color:#c9d1d9}.hljs-doctag,.hljs-keyword,.hljs-meta .hljs-keyword,.hljs-template-tag,.hljs-template-variable,.hljs-type,.hljs-variable.language_{color:#ff7b72}.hljs-title,.hljs-title.class_,.hljs-title.class_.inherited__,.hljs-title.function_{color:#d2a8ff}.hljs-attr,.hljs-attribute,.hljs-literal,.hljs-meta,.hljs-number,.hljs-operator,.hljs-selector-attr,.hljs-selector-class,.hljs-selector-id,.hljs-variable{color:#79c0ff}.hljs-meta .hljs-string,.hljs-regexp,.hljs-string{color:#a5d6ff}.hljs-built_in,.hljs-symbol{color:#ffa657}.hljs-code,.hljs-comment,.hljs-formula{color:#8b949e}.hljs-name,.hljs-quote,.hljs-selector-pseudo,.hljs-selector-tag{color:#7ee787}.hljs-subst{color:#c9d1d9}.hljs-section{color:#1f6feb;font-weight:700}.hljs-bullet{color:#f2cc60}.hljs-emphasis{color:#c9d1d9;font-style:italic}.hljs-strong{color:#c9d1d9;font-weight:700}.hljs-addition{background-color:#033a16;color:#aff5b4}.hljs-deletion{background-color:#67060c;color:#ffdcd7}.hljs-copy-wrapper{overflow:hidden;position:relative}.hljs-copy-button:focus,.hljs-copy-wrapper:hover .hljs-copy-button{transform:translateX(0)}.hljs-copy-button{background-color:#2d2b57;background-color:var(--hljs-theme-background);background-image:url('data:image/svg+xml;charset=utf-8,<svg xmlns="http://www.w3.org/2000/svg" width="16" height="16" fill="none" viewBox="0 0 24 24"><path fill="%23fff" fill-rule="evenodd" d="M6 5a1 1 0 0 0-1 1v14a1 1 0 0 0 1 1h12a1 1 0 0 0 1-1V6a1 1 0 0 0-1-1h-2a1 1 0 1 1 0-2h2a3 3 0 0 1 3 3v14a3 3 0 0 1-3 3H6a3 3 0 0 1-3-3V6a3 3 0 0 1 3-3h2a1 1 0 0 1 0 2H6Z" clip-rule="evenodd"/><path fill="%23fff" fill-rule="evenodd" d="M7 3a2 2 0 0 1 2-2h6a2 2 0 0 1 2 2v2a2 2 0 0 1-2 2H9a2 2 0 0 1-2-2V3Zm8 0H9v2h6V3Z" clip-rule="evenodd"/></svg>');background-position:50%;background-repeat:no-repeat;border:1px solid #ffffff22;border-radius:.25rem;color:#fff;height:2rem;position:absolute;right:1em;text-indent:-9999px;top:1em;transition:background-color .2s ease,transform .2s ease-out;width:2rem}.hljs-copy-button:hover{border-color:#ffffff44}.hljs-copy-button:active{border-color:#ffffff66}.hljs-copy-button[data-copied=true]{background-image:none;text-indent:0;width:auto}@media (prefers-reduced-motion){.hljs-copy-button{transition:none}}.hljs-copy-alert{clip:rect(0 0 0 0);-webkit-clip-path:inset(50%);clip-path:inset(50%);height:1px;overflow:hidden;position:absolute;white-space:nowrap;width:1px}
/*! tailwindcss v3.3.2 | MIT License | https://tailwindcss.com*/*,:after,:before{border:0 solid #e5e7eb;box-sizing:border-box}:after,:before{--tw-content:""}html{-webkit-text-size-adjust:100%;font-feature-settings:normal;font-family:ui-sans-serif,system-ui,-apple-system,BlinkMacSystemFont,Segoe UI,Roboto,Helvetica Neue,Arial,Noto Sans,sans-serif,Apple Color Emoji,Segoe UI Emoji,Segoe UI Symbol,Noto Color Emoji;font-variation-settings:normal;line-height:1.5;-moz-tab-size:4;-o-tab-size:4;tab-size:4}body{line-height:inherit;margin:0}hr{border-top-width:1px;color:inherit;height:0}abbr:where([title]){-webkit-text-decoration:underline dotted;text-decoration:underline dotted}h1,h2,h3,h4,h5,h6{font-size:inherit;font-weight:inherit}a{color:inherit;text-decoration:inherit}b,strong{font-weight:bolder}code,kbd,pre,samp{font-family:ui-monospace,SFMono-Regular,Menlo,Monaco,Consolas,Liberation Mono,Courier New,monospace;font-size:1em}small{font-size:80%}sub,sup{font-size:75%;line-height:0;position:relative;vertical-align:baseline}sub{bottom:-.25em}sup{top:-.5em}table{border-collapse: